In [1]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True) 
import jieba
import random
import os
from config import *

## 加载源数据

In [2]:
technology = pd.read_csv(technology_news_file, encoding='utf-8')
technology.shape
technology.head()

technology.isnull().any()
technology.dropna(inplace=True)
technology.shape

(25482, 2)

,Unnamed: 0,content
0,0,",中新网,1月7日电 恰逢CES 2017拉开大幕，却惊闻“AlphaGo升级版”的M..."
1,1,徐立，商汤科技CEO在谈起本次参展时谈到：“作为一个成立刚刚两年的创业公司，这次参展，一...
2,2,正如最后挑战Master的古力在落败后发表的观点：“人类与人工智能共同探索围棋世界的大幕...
3,3,", SenseFace人脸布控的“黑科技”"
4,4,本届大展最大的看点，无疑是“被誉为2016全美科技第一神股”英伟达的首次CES主题演讲。...


Unnamed: 0    False
content        True
dtype: bool

(25114, 2)

In [3]:
car = pd.read_csv(car_news_file, encoding='utf-8')
car.shape
car.head()

car.isnull().any()
car.dropna(inplace=True)
car.shape

(11959, 2)

,Unnamed: 0,content
0,0,本报济南1月2日电 (记者潘俊强)日前，备受关注的济南“专车案”一审宣判。济南市市中区人...
1,1,2015年1月，陈超在使用专车软件开“专车”送客时，被济南市客管中心认定为非法运营的“黑...
2,2,济南市市中区人民法院审理认为，本案中，陈超在与乘客通过网络约车软件取得联系后，使用未取得...
3,3,同时，虽然被告对未经许可擅自从事出租汽车客运的行为可以依法进行处罚，但原告在本案所涉道路...
4,4,本案宣判后，原告当庭表示不上诉，被告未当庭表示是否上诉。


Unnamed: 0    False
content        True
dtype: bool

(11779, 2)

In [4]:
entertainment = pd.read_csv(entertainment_news_file, encoding='utf-8')
entertainment.shape
entertainment.head()

entertainment.isnull().any()
entertainment.dropna(inplace=True)
entertainment.shape

(40201, 2)

,Unnamed: 0,content
0,0,2016年是综艺井喷的一年，《2016年中国网络视听发展研究报告》数据显示，截至2016...
1,1,区别
2,2,平台决定了资源的多寡和资本的投入
3,3,网络综艺和电视综艺最大的区别在哪？其实是平台。因为平台决定了资源的多寡和资本的投入。所以...
4,4,网络综艺与电视综艺在播出模式、观众群体以及节目板块等方面也都存在差异。在传播上，电视台比...


Unnamed: 0    False
content        True
dtype: bool

(39353, 2)

In [5]:
military = pd.read_csv(military_news_file, encoding='utf-8')
military.shape
military.head()

military.isnull().any()
military.dropna(inplace=True)
military.shape

(17225, 2)

,Unnamed: 0,content
0,0,新华社北京1月8日电(记者董瑞丰)军民融合发展已经上升为国家战略，但现阶段国防工业企业军...
1,1,中国国防工业企业军民融合产业联盟8日在北京宣布成立。刘东奎说，适时成立联盟，坚持“军民融...
2,2,数据显示，我国目前有十几万家民营高新技术企业，很多高新技术企业在新能源、新材料、电子信息...
3,3,对于现阶段军民融合面临的问题，刘东奎列举：从企业外部来看，军民融合管理体制不够顺畅，军民...
4,4,“国防科技工业是军民融合的天然载体和核心领域。”刘东奎说，未来要着力搭建军民协同创新展示...


Unnamed: 0    False
content        True
dtype: bool

(16816, 2)

In [6]:
sports = pd.read_csv(sports_news_file, encoding='utf-8')
sports.shape
sports.head()

sports.isnull().any()
sports.dropna(inplace=True)
sports.shape

(33046, 2)

,Unnamed: 0,content
0,0,",中新网,深圳1月2日电（记者 郑小红）2017年深圳网球公开赛单打正赛1日在深圳市龙岗..."
1,1,1日的首场比赛中，世界排名38位的尼库莱斯库面临了来自中国球员张恺琳的巨大挑战。开局不利...
2,2,参赛球员中排名最高的中国球员王蔷当日表现神勇，用时74分钟便以6-2，6-3横扫俄罗斯球...
3,3,王蔷今年24岁，来自天津，因其纯真的性格颇受欢迎。本场比赛她充分利用球场调动对手的跑动，...
4,4,四名从资格赛中突围成功的球员也已产生，她们是突尼斯球员加布尔、中华台北球员张凯贞、塞尔维...


Unnamed: 0    False
content        True
dtype: bool

(32774, 2)

## 分词去停用词

In [7]:
# 加载停用词
stopwords = pd.read_csv(stopwords_file, index_col=False, quoting=3,sep="\t", names=['stopword'], encoding='utf-8')
stopwords = stopwords['stopword'].values

# 文本预处理
def preprocess_content(content, category):
    out_f = open(processed_data_path + category + ".txt", 'w', encoding="utf-8") # 新建文件
    for line in content:
        try:
            segs=jieba.lcut(line)
            segs = list(filter(lambda x:len(x) > 1, segs)) #没有解析出来的内容过滤掉
            segs = list(filter(lambda x:x not in stopwords, segs)) #把停用词过滤掉
            out_f.write(" ".join(segs)+"\n")
        except Exception as e:
            print(line)
            continue
    out_f.close()

In [8]:
technology_content = technology.content.values.tolist()[1000:6000]
car_content = car.content.values.tolist()[1000:6000]
entertainment_content = entertainment.content.values.tolist()[:5000]
military_content = military.content.values.tolist()[:5000]
sports_content = sports.content.values.tolist()[:5000]
car_content[:2]

['\u3000\u3000全新途观L搭载4MOTION智能全时四驱，采用Haldex第五代中央差速器，实现全时四驱的卓越驾控，帮助其面对崎岖道路亦能如履平地。不仅如此，全新途观L还提供7种驾驶模式选择，可在4种公路模式和3种越野模式之间随心切换，轻松胜任各种路况。前者提供经济、普通、运动、自定义四种不同的驾驶模式可调，令驾驶者尽享铺装路面的驾驶乐趣；后者分为雪地、越野、越野自定义三种模式。雪地四驱模式下扭矩输出更为温柔、平稳，冰雪路面起步更加顺利；越野四驱模式时，EDS功能、下坡辅助、上坡辅助和驻车辅助功能均可帮助驾驶者控制行进轨迹，严防车辆打滑失控；而越野四驱自定义模式可根据个人喜好来设置专属自己的越野模式。',
 '\u3000\u3000除此以外，全新途观L的悬架结构在经过重新设计后，塑造出运动性底盘和舒适性底盘融于一体的经典德系风格，更加适合国内复杂路况及用户使用习惯，实现灵动舒适的驾驭体验。EPS电子助力转向系统的运用，确保驾驶员始终获得清晰路感，也使前行之路更为畅快。同时，全新途观L拥有多达52处静音技术处理，以高标准构建极致静谧空间，让出行品质加倍提升。']

In [9]:
if not os.path.exists(os.path.join(processed_data_path, 'technology.txt') and
                          os.path.join(processed_data_path, 'car.txt') and
                          os.path.join(processed_data_path, 'entertainment.txt') and
                          os.path.join(processed_data_path, 'military.txt') and
                          os.path.join(processed_data_path, 'sports.txt')):
    
    print('分词去停用词...')
    preprocess_content(technology_content, 'technology')
    preprocess_content(car_content,'car')
    preprocess_content(entertainment_content, 'entertainment')
    preprocess_content(military_content, 'military')
    preprocess_content(sports_content, 'sports')
    print('文本预处理已完成！')

## 加载预处理过后的文本数据

In [10]:
# 读取多个文件，返回内容和标签
def read_files(dirname):
    contents = []
    labels = []
    files = [f for f in os.listdir(dirname) if f.endswith(".txt")]
    # print(files)  # ['car.txt', 'entertainment.txt', 'military.txt', 'sports.txt', 'technology.txt']
    for filename in files:
        # print(os.path.join(dirname, filename)) # ./processed_data/car.txt
        content, label = read_single_file(os.path.join(dirname, filename))
        contents.extend(content)
        labels.extend([label] * len(content))
    return contents, labels


# 读取单个txt文件
def read_single_file(filename):
    contents = []
    # label = filename.split('\\')[-1].split('.')[0]
    label = filename.split('/')[-1].split('.')[0]
    with open(filename, mode='r', encoding='utf-8') as f:
        for line in f:
            try:
                content = line.strip().split(' ')
                if content:
                    contents.append(content)
            except:
                pass
    return contents, label


contents, labels = read_files(processed_data_path)
print(contents[:2])
print(labels[:2])

[['全新', '途观', '搭载', '4MOTION', '智能', '全时', '四驱', '采用', 'Haldex', '第五代', '中央', '差速器', '全时', '四驱', '卓越', '驾控', '面对', '崎岖', '道路', '如履平地', '不仅如此', '全新', '途观', '提供', '驾驶', '模式', '选择', '公路', '模式', '越野', '模式', '之间', '随心', '切换', '轻松', '胜任', '路况', '提供', '经济', '运动', '自定义', '四种', '驾驶', '模式', '可调', '驾驶者', '尽享', '路面', '驾驶', '乐趣', '分为', '雪地', '越野', '越野', '自定义', '三种', '模式', '雪地', '四驱', '模式', '扭矩', '输出', '温柔', '平稳', '冰雪', '路面', '起步', '顺利', '越野', '四驱', '模式', 'EDS', '功能', '下坡', '辅助', '上坡', '辅助', '驻车', '辅助', '功能', '驾驶者', '控制', '行进', '轨迹', '严防', '车辆', '打滑', '失控', '越野', '四驱', '自定义', '模式', '喜好', '设置', '专属', '越野', '模式'], ['全新', '途观', '悬架', '结构', '设计', '塑造出', '运动性', '底盘', '舒适性', '底盘', '融于', '一体', '经典', '德系', '风格', '适合', '国内', '路况', '用户', '习惯', '灵动', '舒适', '驾驭', '体验', 'EPS', '电子', '助力', '转向', '系统', '确保', '驾驶员', '始终', '清晰', '路感', '前行', '畅快', '全新', '途观', '拥有', '多达', '静音', '技术', '高标准', '构建', '极致', '静谧', '空间', '出行', '品质', '加倍', '提升']]
['car', 'car']


In [11]:
data = list(zip(contents, labels))
# 乱序
random.shuffle(data)
print(data[:2])

[(['暂停', '加油机'], 'car'), (['备受瞩目', '虚拟现实', '俨然', 'CES', '标配', 'VR', '头显', '更是', '随处可见', '要说', 'CES', 'VR', '头显', '共同点', '支持', '位置', '追踪', '必占', '一席', 'inside', '追踪', '方案', '业界', '热衷', '方向', '英特尔', '高通', '联想', '国内外', '巨头', '纷纷', '推出', 'inside', '相关', '头显', '解决方案'], 'technology')]


In [12]:
contents, labels = zip(*data)
print(contents[:2])
print(labels[:2])

(['暂停', '加油机'], ['备受瞩目', '虚拟现实', '俨然', 'CES', '标配', 'VR', '头显', '更是', '随处可见', '要说', 'CES', 'VR', '头显', '共同点', '支持', '位置', '追踪', '必占', '一席', 'inside', '追踪', '方案', '业界', '热衷', '方向', '英特尔', '高通', '联想', '国内外', '巨头', '纷纷', '推出', 'inside', '相关', '头显', '解决方案'])
('car', 'technology')


## 构建词典

In [13]:
from collections import Counter

def build_vocab(dirname, min_count = 3):
    contents, _ = read_files(dirname)

    words = []
    for word in contents:
        words.extend(word)
        
    counter = Counter(words)
    counter_list = counter.most_common()  # 列出前n个词 [('中国', 4375), ('汽车', 2683),...]
    # 过滤掉低频词
    counter_list = list(filter(lambda x: x[1] > min_count, counter_list))
    words, _ = list(zip(*counter_list))
    word2index = dict(zip(words, range(len(words))))
    index2word = dict([(v, k) for k, v in word2index.items()])
    return word2index, index2word

word2index, index2word = build_vocab(processed_data_path)

## 构建训练数据集

In [14]:
categories = ['car', 'entertainment', 'military', 'sports', 'technology']
cate2idx = dict(zip(categories, range(len(categories))))
cate2idx

def encode_label(labels, cate2idx):
    return [(cate2idx[label] if label in cate2idx else 0) for label in labels]

y = encode_label(labels, cate2idx)
y[:5]

{'car': 0, 'entertainment': 1, 'military': 2, 'sports': 3, 'technology': 4}

[0, 4, 0, 3, 3]

In [15]:
def build_dataset(x, y, word2index, maxlen=100):
    dataset = []
    new_labels = []
    arr = np.arange(len(x)) # arange函数用于创建等差数组
    for i in range(arr.shape[0]):
        new_labels.append(y[i])

        new_line = []
        for word in x[i]:
            if word in word2index:
                index = word2index[word]
            else:
                index = 0  # <UNK>
            new_line.append(index)
            

        # padding
        pad_num = maxlen - len(new_line)
        while pad_num > 0:
            new_line.append(0)  # <PAD>
            pad_num -= 1

        dataset.append(new_line[:maxlen])

    return np.asarray(dataset), np.asarray(new_labels)


x,y = build_dataset(contents, y, word2index)
x.shape
y.shape

(25072, 100)

(25072,)

In [16]:
x[:2]
y[:2]

array([[ 2882,  9427,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [ 8922,  2791,  6907,   638,  1903,   290, 10177,   175,  8222,
         5700,   638,   290, 10177, 15973,   137,   467,  4250,     0,
        18560,  8169,  4250,   443,  1203, 15910,   394,  234

array([0, 4])

## 模型构建

In [17]:
from tensorflow.keras import Input, Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, Lambda, Concatenate, Conv1D, GlobalMaxPooling1D


class RCNN(object):
    def __init__(self, maxlen, max_features, embedding_dims, class_num=5, last_activation='softmax'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input_current = Input((self.maxlen,))
        input_left = Input((self.maxlen,))
        input_right = Input((self.maxlen,))

        embedder = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)
        embedding_current = embedder(input_current)
        embedding_left = embedder(input_left)
        embedding_right = embedder(input_right)

        x_left = SimpleRNN(128, return_sequences=True)(embedding_left)
        x_right = SimpleRNN(128, return_sequences=True, go_backwards=True)(embedding_right)
        x_right = Lambda(lambda x: K.reverse(x, axes=1))(x_right)
        x = Concatenate(axis=2)([x_left, embedding_current, x_right])

        x = Conv1D(64, kernel_size=1, activation='tanh')(x)
        x = GlobalMaxPooling1D()(x)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=[input_current, input_left, input_right], outputs=output)
        return model

## 模型训练

In [18]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


for train_index, test_index in StratifiedKFold(n_splits=3, random_state=2019).split(x, y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    print('为模型准备输入数据...')
    x_train_current = x_train
    x_train_left = np.hstack([np.expand_dims(x_train[:, 0], axis=1), x_train[:, 0:-1]])
    x_train_right = np.hstack([x_train[:, 1:], np.expand_dims(x_train[:, -1], axis=1)])
    print('x_train_current 维度:', x_train_current.shape)
    print('x_train_left 维度:', x_train_left.shape)
    print('x_train_right 维度:', x_train_right.shape)
    
    x_test_current = x_test
    x_test_left = np.hstack([np.expand_dims(x_test[:, 0], axis=1), x_test[:, 0:-1]])
    x_test_right = np.hstack([x_test[:, 1:], np.expand_dims(x_test[:, -1], axis=1)])
    print('x_test_current 维度:', x_test_current.shape)
    print('x_test_left 维度:', x_test_left.shape)
    print('x_test_right 维度:', x_test_right.shape)
    
    # 将整型标签转为onehot
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
          
    print('构建模型...')
    max_features = len(word2index) + 1
    model = RCNN(maxlen, max_features, embedding_dims).get_model()
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

    print('训练...')
    my_callbacks = [
        ModelCheckpoint('./rcnn_model.h5', verbose=1),
        EarlyStopping(monitor='val_accuracy', patience=2, mode='max')
    ]

    # fit拟合数据
    history = model.fit([x_train_current, x_train_left, x_train_right], y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=my_callbacks,
              validation_data=([x_test_current, x_test_left, x_test_right], y_test))

为模型准备输入数据...
x_train_current 维度: (16713, 100)
x_train_left 维度: (16713, 100)
x_train_right 维度: (16713, 100)
x_test_current 维度: (8359, 100)
x_test_left 维度: (8359, 100)
x_test_right 维度: (8359, 100)
构建模型...
训练...
Train on 16713 samples, validate on 8359 samples
16640/16713 [============================>.] - ETA: 0s - loss: 1.5279 - accuracy: 0.2942
Epoch 00001: saving model to ./rcnn_model.h5
16713/16713 [==============================] - 55s 3ms/sample - loss: 1.5281 - accuracy: 0.2945 - val_loss: 1.4880 - val_accuracy: 0.3663
为模型准备输入数据...
x_train_current 维度: (16714, 100)
x_train_left 维度: (16714, 100)
x_train_right 维度: (16714, 100)
x_test_current 维度: (8358, 100)
x_test_left 维度: (8358, 100)
x_test_right 维度: (8358, 100)
构建模型...
训练...
Train on 16714 samples, validate on 8358 samples
16640/16714 [============================>.] - ETA: 0s - loss: 1.4495 - accuracy: 0.3269
Epoch 00001: saving model to ./rcnn_model.h5
16714/16714 [==============================] - 59s 4ms/sample - loss: 1.4492 -

## 模型预测

In [20]:
result = model.predict([x_test_current, x_test_left, x_test_right])
result

array([[0.17690817, 0.18487675, 0.20775864, 0.23029561, 0.20016076],
       [0.15817654, 0.16677041, 0.2262304 , 0.28463072, 0.16419196],
       [0.13460504, 0.17359997, 0.25045228, 0.3032397 , 0.13810301],
       ...,
       [0.16007912, 0.19630821, 0.21588951, 0.25138694, 0.17633626],
       [0.1775948 , 0.19428393, 0.21069928, 0.25095233, 0.1664697 ],
       [0.41847914, 0.25847763, 0.07134484, 0.04209331, 0.20960507]],
      dtype=float32)